In [1]:
import sys
sys.path.append('..\..')
import torch
from torch import nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
%matplotlib notebook
from ipywidgets import *
import matplotlib.pyplot as plt
import plotly.express as px
from utils.functions import train_models, train_models_w_mean_var, train_and_save_results
from utils.models import BatchEnsemble, AnchoredBatchEnsemble
from utils.layers import BatchLinear
from utils.layers import AnchoredBatch

## Load and Process Concrete Dataset

In [2]:
# Read data into a pandas dataframe
concrete = pd.read_excel('..\\..\\data\\UCI_Regression\\2.Concrete\\Concrete_Data.xls')

concrete.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [3]:
# Set a seed for reproducibility
np.random.seed(52)

# Split into training and validation datasets
concrete_train, concrete_val = train_test_split(concrete, test_size=0.1)
concrete_train, concrete_val = concrete_train.to_numpy(), concrete_val.to_numpy()

In [4]:
# Split the dataset into features and labels
X = concrete_train[:, :8]
y = concrete_train[:, 8]

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

# Subtract mean and divide by standard deviation for the training set
scaler_train = StandardScaler()
x_train = scaler_train.fit_transform(x_train)

# Use a new scaler to normalize the test set
scaler_test = StandardScaler()
x_test = scaler_test.fit_transform(x_test)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert numpy arrays to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)

print(x_train_tensor.shape)
print(y_train_tensor.shape)
print(x_test_tensor.shape)
print(y_test_tensor.shape)

# Create TensorDatasets
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

# Define a batch size
batch_size = 128

# Create DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


torch.Size([648, 8])
torch.Size([648, 1])
torch.Size([279, 8])
torch.Size([279, 1])


## Hyperparameter tuning

In [5]:
# Initialize list of Hyperparameters for Anchored Batch Ensemble
hidden_layers_options = [2,4,6,8,10]
hidden_units_options = [64,128,256]
dropout_prob_options = [0,0.2,0.5]
data_noise_options = [1e-2, 1e-4,1e-6]

# Define loss function and optimizer
loss_fn = nn.GaussianNLLLoss()
optimizer = torch.optim.Adam

# Train and save results
train_and_save_results(
    model_name = 'anchored_batch', 
    hidden_layers_options = hidden_layers_options, 
    hidden_units_options = hidden_units_options, 
    input_shape = 8, 
    loss_fn = loss_fn,
    optimizer = optimizer, 
    train_loader = train_loader, 
    test_loader = test_loader,
    ensemble_size = 16,
    epochs = 2000, 
    csv_file = '..\\concrete_model_results.csv',
    tensorboard_directory = 'concrete_anchored',
    dropout_prob_options=dropout_prob_options,
    early_stopping=False,
    data_noise_options = data_noise_options,
    print_frequency = 50
)

Epoch: 0
-------
Train Loss: 187512.240234375
Epoch: 50
-------
Train Loss: 306.17230733235675
Epoch: 100
-------
Train Loss: 11.99587074915568
Epoch: 150
-------
Train Loss: 7.619416077931722
Epoch: 200
-------
Train Loss: 6.311541398366292
Epoch: 250
-------
Train Loss: 5.951773325602214
Epoch: 300
-------
Train Loss: 5.835366408030192
Epoch: 350
-------
Train Loss: 5.464043219884236
Epoch: 400
-------
Train Loss: 5.321397463480632
Epoch: 450
-------
Train Loss: 5.241158644358317
Epoch: 500
-------
Train Loss: 5.246258894602458
Epoch: 550
-------
Train Loss: 5.062218030293782
Epoch: 600
-------
Train Loss: 5.033875306447347
Epoch: 650
-------
Train Loss: 5.2130982875823975
Epoch: 700
-------
Train Loss: 5.106974919637044
Epoch: 750
-------
Train Loss: 4.981157461802165
Epoch: 800
-------
Train Loss: 4.924009958902995
Epoch: 850
-------
Train Loss: 4.754287560780843
Epoch: 900
-------
Train Loss: 4.908973852793376
Epoch: 950
-------
Train Loss: 4.826608260472615
Epoch: 1000
-------
Tr